In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
provider = SaverProvider()

In [9]:
from Scratch import metadata_loader

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.DONE', 'JobStatus.ERROR', 'JobStatus.CANCELLED'])]
df

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr,execution_date


In [10]:
from time import sleep

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=False)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  


0it [00:00, ?it/s]

In [13]:
memory = provider.retrieve_job('cnp4xn60k0p00088wmxg').result().get_memory()
print(memory.shape)

(1, 5556)


# update the metadata with the execution date

In [12]:
from Scratch import update_metadata

df = metadata_loader(_extract=True)
df = df[~df['job_status'].isin(['JobStatus.ERROR', 'JobStatus.CANCELLED'])]

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            ibm_job = provider.retrieve_job(job_id, _ignore_running=False)
            additional_dict = {"execution_date": str(ibm_job.result().date)}
            update_metadata(ibm_job, "None_Backend", additional_dict)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  
            

  0%|          | 0/1179 [00:00<?, ?it/s]